In [35]:
import tweepy as tw
import pandas as pd
import pickle
import plotly.express as plotly
import re
import operator
import statistics
from datetime import datetime
from datetime import timedelta
from configu import credentials

In [36]:
api_key = credentials["api_key"]
sec_key = credentials["sec_key"]
access_token = credentials["access_token"]
access_token_secret = credentials["access_token_secret"]

In [37]:
auth = tw.AppAuthHandler(api_key, sec_key)
auth.secure = True
api = tw.API(auth, wait_on_rate_limit=True)

In [38]:
dp_tweets_list = []
for status in tw.Cursor(api.user_timeline, screen_name="@DelhiPolice", tweet_mode='extended').items(30000):
    dp_tweets_list.append(status)
print(len(dp_tweets_list))

open_file = open("twitter_data.txt", "wb")
pickle.dump(dp_tweets_list, open_file)
open_file.close()

In [39]:
dp_rep = []
for tweet in dp_tweets_list:
    if(tweet.in_reply_to_status_id_str != None):
        dp_rep.append(tweet)

org_tw = []
dp_rep_rel = []

for tweet in dp_rep:
    try:
        ot = api.get_status(tweet.in_reply_to_status_id, tweet_mode = 'extended')
        if("DelhiPolice" in ot.full_text):
            org_tw.append(ot)
            dp_rep_rel.append(tweet)
    except:
        continue
        
print(len(org_tw))
print(len(dp_rep_rel))

In [40]:
open_file = open("org_tweets.txt", "wb")
pickle.dump(org_tw, open_file)
open_file.close()

open_file = open("dp_relevant_rep.txt", "wb")
pickle.dump(dp_rep_rel, open_file)
open_file.close()

open_file = open("twitter_data.txt", "rb")
dp_list = pickle.load(open_file)
open_file.close()

open_file = open("org_tweets.txt", "rb")
org_t = pickle.load(open_file)
open_file.close()

open_file = open("dp_relevant_rep.txt", "rb")
dp_rep = pickle.load(open_file)
open_file.close()

In [ ]:
mail_reg = "(\w|\.|\_|\-)+[@](\w|\_|\-|\.)+[.]\w{2,3}"
reg1 = re.compile(mail_reg)

no_reg = "\+?\d[\d -]{8,12}\d"
reg2 = re.compile(no_reg)

aadh_reg = "[2-9]{1}[0-9]{3}\\s[0-9]{4}\\s[0-9]{4}"
reg3 = re.compile(aadh_reg)

pan_reg = "[A-Z]{5}[0-9]{4}[A-Z]{1}"
reg4 = re.compile(pan_reg)

master_reg = "5[1-5][0-9]{14}|^(222[1-9]|22[3-9]\\d|2[3-6]\\d{2}|27[0-1]\\d|2720)[0-9]{12}"
reg5 = re.compile(master_reg)

car_reg = "[A-Z]{2}[ -][0-9]{1,2}(?: [A-Z])?(?: [A-Z]*)? [0-9]{4}"
reg6 = re.compile(car_reg)

regex = [reg1, reg2, reg3, reg4, reg5, reg6]

no_mail = 0
no_phn = 0
no_car = 0
for tweet in org_t:
    for reg in regex:
        if(re.search(reg, tweet.full_text) and "media" not in tweet.entities.keys()):
            if(reg == reg1):
                no_mail += 1
            elif(reg == reg2):
                no_phn += 1
            elif(reg == reg6):
                no_car += 1
                
print("Email:",no_mail)
print("Phone Numbers:",no_phn)
print("Vehicle Numbers:",no_car)

In [ ]:
media = 0
pii_med = 0
mail_med = 0
phn_med = 0
car_med = 0
for tweet in org_t:
    if("media" in tweet.entities.keys()):
        media += 1
        for reg in regex:
            if(re.search(reg, tweet.full_text)):
                if(reg == reg1):
                    mail_med += 1
                elif(reg == reg2):
                    phn_med += 1
                elif(reg == reg6):
                    car_med += 1
                pii_med += 1
    
print("Tweets containing media =",media)
print("Tweets containg PII & media =",pii_med)
print("Email:",mail_med)
print("Phone Numbers:",phn_med)
print("Vehicle Numbers:",car_med)

In [43]:
tw_dict = {}
org_tweet = []
dp_reply = []
for tweet in org_t:
    for rep in dp_rep:
        if(rep.in_reply_to_status_id == tweet.id and tweet.user.screen_name != "DelhiPolice"):
            tw_dict[tweet.id] = rep.in_reply_to_status_id
            org_tweet.append(tweet)
            dp_reply.append(rep)

In [44]:
resp_time = []
rt_dict = {}
for i in range(len(dp_reply)):
    rt = dp_reply[i].created_at - org_tweet[i].created_at
    rt_sec = rt.total_seconds()
    resp_time.append(rt_sec)
    rt_dict[org_tweet[i].created_at] = rt_sec/(60*60)

In [ ]:
min_time = timedelta(seconds = min(resp_time))
max_time = timedelta(seconds = max(resp_time))
avg_time = timedelta(seconds = statistics.mean(resp_time))
std_dev = timedelta(seconds = statistics.pstdev(resp_time))
print("Minimum response time by @DelhiPolice:",min_time)
print("Maximum response time by @DelhiPolice:",max_time)
print("Average response time by @DelhiPolice:",avg_time)
print("Standard deviation of response time by @DelhiPolice:",std_dev)

In [ ]:
#Timeseries Plot

rt_dict = dict(sorted(rt_dict.items(), key=operator.itemgetter(0)))
resp_data = pd.DataFrame(list(zip(rt_dict.keys(), rt_dict.values())), columns =['date', 'resp_time'])

figure = plotly.line(x=resp_data.date, y=resp_data.resp_time, labels={'x':'Date', 'y':'Response Time (hours)'})
figure.show()